<a href="https://colab.research.google.com/github/alexiamhe93/RAMP_method/blob/main/RAMP_Python_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Recursive Adjustment of Measurement Protocols (RAMP) method: Case study code for replication




The notebook was designed using Google Colab on an Nvidia T4 GPU (free with log-in). The code works locally but all dependencies from the "Load packages" will have to be installed.

This Python notebook is used to replicate the results of the paper titled:

"Recursive Adjustment of Measurement Protocols (RAMP) method for developing high-validity text classifiers"

The notebook is structured in terms of the RAMP stages:
0. Install and load data / model for analysis

1. Manual coding stage. This notebook runs inter-rater reliability statistics on the final shared subset of data.

2. Computation stage. Uses the coded dataset to develop three different text classifiers (rule-based, supervised machine learning, LLM few-shot).

3. Evaluation stage: Identify and evaluate surprises and outliers in classifier development, with the goal of identifying construct and content validity issues. The code for this section prints the manual coding disagreements and classifiers.

> Each stage is structured in three phases, an input (defines the parameters), a throughput (development stage), and an output (final validation).
____________________________

The notebook applies RAMP to a case study on measuring misunderstandings in online dialogue data.


## 0. Initiate notebook

This section installs all the necessary Python packages to complete these analysis. We also download the data and pre-trained BERT model for replicating the results.

The rule-based classifier

--------------------
A few packages require mention as they are non-standard:

> spacy ([Honnibal et al., 2022](https://github.com/explosion/spaCy)).

This package is used for creating a rule-based dictionary classifier, similar to LIWC ([Pennebaker et al., 2001](http://downloads.liwc.net.s3.amazonaws.com/LIWC2015_OperatorManual.pdf)). This

> ktrain ([Maiya, 2022](https://github.com/amaiya/ktrain)).

This package is a Keras wrapper for streamlining many tasks related to fine-tuning and deploying deep learning models. In this notebook we use it to fine-tune Google's BERT ([Devlin et al., 2019](https://arxiv.org/abs/1810.04805)) base model.

> eli5 ([Korobov, 2017](https://av.tib.eu/media/33771);[Korobov & Lopuhin, 2024](https://github.com/eli5-org/eli5)).

"Explain like I'm five" is a package used for running the LIME ([Ribeiro et al., 2016](http://arxiv.org/abs/1602.04938)) algorithm to examine how a supervised classifier is making its predictions.

> openai ([OpenAI et al., 2024](https://platform.openai.com/docs/api-reference/introduction
 )).

This package accesses the OpenAI API for using GPT-4o within the notebook.

> textstat ([Shivan & Chaitanya, 2024](https://pypi.org/project/textstat/)).

This package calculates simple statistical information relating to raw text data.


## 0.1 Install and load packages

In [ ]:
!pip install tf-keras
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [ ]:
# For Supervised classifier
!pip install ktrain
# For revealing under the classifier black box
!pip install https://github.com/amaiya/eli5-tf/archive/refs/heads/master.zip
# For LLM classifier
!pip install openai
# For summary statistics
!pip install textstat
# This is a port from Gwet's R package with the same name
!pip install irrCAC

In [2]:
# General use packages
import requests, zipfile, io, os, psutil, random, time
import torch
import pandas as pd
# This deactivates a warning from Pandas that frequently prints
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from collections import Counter
from tqdm import tqdm
# For descriptive statistics
from textstat.textstat import textstatistics
import re
# Performance evaluations for binary classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve, auc, roc_auc_score, matthews_corrcoef
# For calculating inter-rater reliability
from irrCAC.raw import CAC
#for troubleshooting
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
from nltk import agreement
import matplotlib.pyplot as plt

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Plotting
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 120

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [3]:
# for rule-based classification
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_sm")

In [ ]:
# for supervised classification
import ktrain
from ktrain import text
# for LLM classification
import openai

In [ ]:
# Check system GPU (recommended if possible)
# CPU cores
num_cpu_cores = os.cpu_count()
print(f"Number of CPU cores: {num_cpu_cores}")
# GPU details
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)  # in GB
    print(f"GPU Name: {gpu_name}")
    print(f"GPU Memory: {gpu_memory:.2f} GB")
else:
    print("No GPU available.")

Number of CPU cores: 2
No GPU available.


In [ ]:
# Load in openai keys for few-shot classifier
oai_k = "your-API-key-here"
openai.organization = "your-organization-key-here" #if applicable
openai.api_key = oai_k
os.environ['OPENAI_API_KEY'] = oai_k

## 0.2 Download data and pre-trained BERT model

All the data for replication (<50mb) is accessed through a GitHub link and the pre-trained BERT model (1.03GB) from dropbox

Download data from GitHub

In [4]:
# Download empirical data
r = requests.get('https://github.com/alexiamhe93/RAMP_method/blob/main/Dataset/data.zip?raw=true')
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

# Load train (70%) and test (30%)
try:
  train = pd.read_csv("data/Train.csv")
  validation = pd.read_csv("data/Validation.csv")
  St1Through = pd.read_csv("data/RAMP_Stage1.csv")
  St2Through = pd.read_csv("data/RAMP_Stage2.csv")
  out = pd.read_csv("data/RAMP_Stage3.csv")
except:
  train = pd.read_csv("Train.csv")
  validation = pd.read_csv("Validation.csv")
  St1Through = pd.read_csv("RAMP_Stage1.csv")
  St2Through = pd.read_csv("RAMP_Stage2.csv")
  out = pd.read_csv("RAMP_Stage3.csv")

In [14]:
print(f"Validation n before cleaning: {len(validation)} texts")
# Delete any duplicates
validation = validation.dropna(subset=["text"])
validation = validation.drop_duplicates(subset="text")
print(f"Validation n after cleaning: {len(validation)} texts")

Validation n before cleaning: 6599 texts
Validation n after cleaning: 6420 texts


We removed 179 sentences that are duplicates or empty values.

Download model from Dropbox (can take some time if internet is slow - aprox 1.1GB - downloads weights and pre-processing)

In [ ]:
!wget -O supervised_model.zip https://www.dropbox.com/scl/fi/5wtuor1ag1gktg6eukqwr/supervised_model.zip?rlkey=nfwxyataobjzt708m3gf27o3s&st=cz5r9lq0&dl=0 --quiet
!unzip supervised_model.zip

## 0.3 Load classes and functios

The notebook uses two class objects for performing most of the operations across the three stages of RAMP. The classifier object is used to calculate inter-rater reliability (manual coding); run a dictionary word classifier (computation), a supervised classifier (computation) and an LLM classifier (computation); calculate accuracy metrics (computation); access disagreements and misclassifications (evaluation).

#### Classifier object and functions

This class does the heavy lifting for the notebook. It integrates the three types of classifier (rule-based, supervised, LLM) into one function so that there is a common language across the examples.

The classifier produces a development report, including the following variables:

1. `TP`,`TN`,`FP`,`FN`: number of true positives, true negatives, false positives, and false negatives
2. `Precision`: TP/TP+FP - ratio of true positives to all predicted positive class. Reported for positive class only.
3. `Recall`: TP/TP+FN – ratio of true positives to all true positive class.Reported for positive class only.
4. `F1_avg`: Weighted harmonic mean of precision and recall (all classes - this F1 is not the precision and recall reported).
5. `F1_var`: Weighted harmonic mean of precision and recall for positive class.
6. `AUC_ROC`: Area under the receiving operating characteristic (ROC) curve
7. `AUC_PR`: Area under the precision and recall curve.
8. `MCC`: Matthews Correlation Coefficient


Each metric highlights a different aspect of the classifier's performance. For instance, the weighted F1 (`F1_avg`) is sensitive to imbalanced classes. For misunderstandings, the class is imbalanced (8% of turns are misunderstandings) so the MCC is more appropriate.

The development report is geared at binary classification and alternative accuracy metrics should be sought for other methods.

In [5]:
class Classifier:
  def __init__(self, texts, true_scores):
    """
    Initialize the Classifier class with texts and true scores.
    """
    self.texts = texts
    self.true_scores = true_scores
    self.train_size = len(texts)
    self.type_ = None
    self.pred_scores = []
    self.nlp = spacy.load("en_core_web_sm")

  def add_pred_scores(self, pred_scores):
    self.pred_scores = pred_scores
  def get_pred_scores(self):
    return self.pred_scores
  def add_rule_based_terms(self, terms, pattern_type):
    """
    Configure terms and pattern matching type for rule-based classifiers.
    """
    self.terms = terms
    if pattern_type == "pattern":
        self.type_ = "rule-based-1"
    elif pattern_type == "lemma":
        self.type_ = "rule-based-2"
    else:
        raise ValueError("Invalid pattern type specified.")
  def classify_with_spacy_pattern(self):
    """
    Classify texts using SpaCy's pattern matcher based on predefined terms.
    """
    matcher = Matcher(self.nlp.vocab)
    for term in self.terms:
        matcher.add(term["label"], [term["pattern"]])
    self.pred_scores = [bool(matcher(self.nlp(text))) for text in self.texts]
  def classify_with_spacy_lemma(self):
    """
    Classify texts by checking if any lemmas from the terms are in the texts.
    """
    lemma_doc = self.nlp(" ".join(self.terms))
    lemma_set = set(token.lemma_ for token in lemma_doc)
    self.pred_scores = [bool(set(token.lemma_ for token in self.nlp(text.lower())) & lemma_set) for text in self.texts]
  def add_SML_classifier(self, predictor, **kwargs):
    """
    Configure the supervised machine learning classifier with a predictor and training parameters.
    """
    self.type_ = "supervised"
    self.predictor = predictor
    self.sml_params = kwargs
    print("Supervised ML classifier configured with parameters:", kwargs)
  def classify_with_SML(self):
    """
    Perform classification using the configured supervised machine learning predictor.
    """
    preds = self.predictor.predict(self.texts)
    self.pred_scores = [0 if "not" in pred.lower() else 1 for pred in preds]

  def add_few_shot_classifier(self, GPTmodel, prompt, role):
    """
    Configure the few-shot classifier with a GPT model, prompt template, and user/system roles.
    """
    self.type_ = "LLM"
    self.GPTmodel = GPTmodel
    self.prompt = prompt
    self.role = role
    self.cost = 0
    self.total_tokens = 0
    self.LLMScores = []

  def gptActualCost(self, response):
    """
    Calculates the GPT cost for different models
    """
    engine = self.GPTmodel
    total_tokens=response.usage.total_tokens
    total_tokens_1k_units = total_tokens/1000

    if engine=='gpt-3.5-turbo':
        cost=total_tokens_1k_units*0.0005
    elif engine=='gpt-4-turbo':
        cost=total_tokens_1k_units*0.01
    elif engine=='gpt-4o':
        cost=total_tokens_1k_units*0.005
    elif engine=='gpt-4-32k':
        cost=total_tokens_1k_units*0.12
    else:
        print('getCost error: engine not found')
        return
    return cost, total_tokens

  def get_llm_response(self,messages,temperature=0, max_tokens = 100, max_attempts = 3):
    '''
    Function that takes messages format for ChatGPT input and returns the response text.
    '''
    GPTmodel = self.GPTmodel
    for attempt in range(0, max_attempts):
      try:
        #. request timeout ADD IN
        response = openai.chat.completions.create(model=GPTmodel, messages = messages, temperature=temperature, max_tokens=max_tokens)
        response_text = response.choices[0].message.content
        self.LLMScores.append(response_text)
        response_cost, token_count = self.gptActualCost(response)
        self.cost += response_cost
        self.total_tokens += token_count
        break  # If analysis was successful, break out of the retry loop
      except Exception as e:
        print(f"Error processing text on attempt {attempt+1}: {e}")
        if attempt + 1 == max_attempts:
          print(f"Skipping text after {max_attempts} failed attempts.")
          response_text
    return response_text
  def define_messages(self, text_to_classify):
    '''
    Function for creating a basic messages format from a prompt, a role, and a text to classify (all strings)
    '''
    prompt = self.prompt
    role = self.role
    prompt = prompt.format(text_to_classify)
    messages = [{'role': 'system', 'content': role},
                {'role': 'user', 'content' : prompt}]
    return messages
  def convert_llm_scores_binary(self, return_scores = False):
    '''
    Function for converting a string "Yes" or "No" into binary format - used for the clarification requests
    '''
    llm_scores = self.pred_scores
    new_scores = []
    for s in llm_scores:
      if "yes" in s.lower():
        new_scores.append(1)
      else:
        new_scores.append(0)
    if not return_scores:
      self.pred_scores = new_scores
    else:
      return new_scores

  def classify_with_fewshot(self,  max_tokens = 100, max_attempts = 3, temperature = 0):
    '''
    Function for running a prompt over a series of texts (expects a list)
    '''
    prompt = self.prompt
    role = self.role
    input_texts = self.texts
    GPTmodel = self.GPTmodel
    scores = []
    for txt in tqdm(input_texts):
      message = self.define_messages(txt)
      try:
        response = self.get_llm_response(message,temperature=temperature,
                                        max_tokens=max_tokens, max_attempts=max_attempts)
      except:
        response = "Error in response"
      scores.append(response)
    self.pred_scores = scores
    self.convert_llm_scores_binary()

  def run_classifier(self):
    """
    Execute the classifier based on the configured type.
    """
    if self.type_ == "rule-based-1":
        self.classify_with_spacy_pattern()
    elif self.type_ == "rule-based-2":
        self.classify_with_spacy_lemma()
    elif self.type_ == "supervised":
        self.classify_with_SML()
    elif self.type_ == "LLM":
        self.classify_with_fewshot()
        cost = self.cost
        total_tokens = self.total_tokens
        avg_tokens = self.total_tokens / self.train_size
        print(f"This run cost {cost:.2f}$ for {total_tokens} tokens. Average tokens: {avg_tokens:.2f}")
    else:
        raise ValueError("Classifier type is not configured.")

  def get_model_report(self, display=True):
    """
    Generate and display or return the classification report and metrics.
    """
    # Generate confusion matrix
    cm = confusion_matrix(self.true_scores, self.pred_scores)
    # Generate classification report
    report = classification_report(self.true_scores, self.pred_scores, output_dict=True)
    # Precision-recall curve and AUC for precision-recall
    precision, recall, thresholds = precision_recall_curve(self.true_scores, self.pred_scores)
    auc_pr = auc(recall, precision)
    # AUC for ROC curve
    auc_roc = roc_auc_score(self.true_scores, self.pred_scores)
    # Matthews Correlation Coefficient (MCC)
    mcc = matthews_corrcoef(self.true_scores, self.pred_scores)

    if display:
        print(f'AUC-PR: {auc_pr:.2f}\n')
        print(f'AUC-ROC: {auc_roc:.2f}\n')
        print(f'MCC: {mcc:.2f}\n')
        print(classification_report(self.true_scores, self.pred_scores, output_dict=False))
    else:
        return {
            "precision": report['1']['precision'],
            "recall": report['1']['recall'],
            "auc_pr": auc_pr,
            "auc_roc": auc_roc,
            "mcc": mcc,
            "f1_avg": report['weighted avg']['f1-score'],
            "f1_var": report['1']['f1-score']
        }
  def get_misclassification(self, return_all = False):
    """
    Function to fetch misclassifications
    """
    df = pd.DataFrame({"text":self.texts,"true":self.true_scores,
                       "pred":self.pred_scores})
    # Function to classify each row
    def classify_row(row):
      if row['true'] == 1 and row['pred'] == 1:
        return 'TP'
      elif row['true'] == 0 and row['pred'] == 1:
        return 'FP'
      elif row['true'] == 1 and row['pred'] == 0:
        return 'FN'
      elif row['true'] == 0 and row['pred'] == 0:
        return 'TN'
    df['Classification'] = df.apply(classify_row, axis=1)
    if return_all:
      return df
    else:
      return df[df["Classification"].isin(["FP","FN"])]

  def preprocess_text(self, text):
    """
    Function to process the texts.
    """
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    words = nltk.word_tokenize(text)
    cleaned_text = [lemmatizer.lemmatize(word.lower()) for word in words if word.isalnum() and word.lower() not in stop_words]
    return ' '.join(cleaned_text)

  def plot_misclassifications(self, df, FN_FP="FN"):
    """
    Function to generate a wordcloud
    """
    df['cleaned_text'] = df['text'].apply(self.preprocess_text)
    if FN_FP == "FN":
      print("Word Cloud for False Negatives:")
      texts = " ".join(df[df['Classification'] == 'FN']['cleaned_text'])
    else:
      print("Word Cloud for False Positives:")
      texts = " ".join(df[df['Classification'] == 'FP']['cleaned_text'])
    wordcloud = WordCloud(width = 800, height = 400, background_color ='white').generate(texts)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

The below class was used to develop the different classifiers. Example usage hashed out below the classifier.

In [ ]:
#import wandb
#import ConfusionMatrixDisplay
def display_confusion_matrix(human_labels, predicted_labels):
  '''
  Function for displaying a confusion matrix from results
  '''
  conf_mx = confusion_matrix(human_labels, predicted_labels)
  disp = ConfusionMatrixDisplay(confusion_matrix=conf_mx)
  disp.plot()


class log_wandb:
  """
  This class logs the classifier training/development runs to a wandb session.
  """

  def __init__(self,wandb_project):
    run = wandb.init(project=wandb_project)
    """
    wandb_project = ID str: name of the wandb project to initiate
    """

  def add_KeyVariables(self,train_size,Precision,Recall,
                       AUC_PR,AUC_ROC,F1_avg,F1_var,
                       TP,TN,FP,FN,type_):

    # variables for all classifier types
    wandb.log({"ClassiTypeStr":type_,
               "train_size":train_size,
               "F1_avg":F1_avg,
               "F1_var":F1_var,
               "AUC_PR":AUC_PR,
               "AUC_ROC":AUC_ROC,
               "Precision":Precision,
               "Recall":Recall,
               "TP":TP,
               "TN":TN,
               "FP":FP,
               "FN":FN})

  def add_ClassVars(self,Terms = None, nTerms=None,PatternOrLemma=None,BERTmodel=None,
                    validation_size=None,learning_rate=None,epochs=None,batch_size=None,
                    GPTmodel=None,role=None,prompt=None,examples=None,nExamples=None,
                    balance_ratio=None):
    # Specific vars to a classifier type
    wandb.log({"Terms":Terms, "nTerms":nTerms,"PatternOrLemma":PatternOrLemma,
               "BERTmodel":BERTmodel,
               "validation_size":validation_size,
               "learning_rate":learning_rate,"epochs":epochs,
               "batch_size":batch_size,"GPTmodel":GPTmodel,
               "role":role,"prompt":prompt,"examples":examples,
               "nExamples":nExamples, "balance_ratio":balance_ratio})

  def end_run(self):
    # Finish wandb session
     wandb.finish()

def summarise_data(df, name = "training"):
  print("----------------------------------------------")
  n_mis = df.Misunderstanding.sum()
  pct_mis = round(n_mis/len(df) * 100)
  print(f"There are {len(df)} sentences in the {name} set.")
  print(f"There are {n_mis} ({pct_mis}%) misunderstandings in the set.")
  print("----------------------------------------------")

class Classifier_Throughput:
  """
  A text classifier that supports rule-based, supervised machine learning,
  and few-shot classification approaches.
  """
  def __init__(self,texts,true_scores):

    """
    Initializes the Classifier with texts and their true classification scores.
    """
    self.texts = texts
    self.true_scores = true_scores
    self.train_size = len(texts)
    self.find_learner = False
    self.type_ = None
    self.patterns = []
    self.lemmas = []
    self.pred_scores = []
    self.LLMScores = []
  # _____________________
  # Rule-based classifiers

  def add_rule_based_terms(self, terms, mode):
    """
    Adds terms for rule-based classification and sets the type of classification based on the mode.
    """
    if mode == "pattern":
      self.patterns = terms
      self.type_ = "rule-based-1"
    elif mode == "lemma":
      self.lemmas = terms
      self.type_ = "rule-based-2"

  def classify_with_spacy_pattern(self, nlp):
    """
    Classifies texts using spaCy's pattern matching for the provided patterns.
    """
    matcher = Matcher(nlp.vocab)
    for pattern in self.patterns:
      matcher.add(pattern["label"], [pattern["pattern"]])

    results = []
    for text in self.texts:
      doc = nlp(text) if text else nlp("notext")
      matches = matcher(doc)
      results.append(len(matches) > 0)
    self.pred_scores = results

  def classify_with_spacy_lemma(self, nlp):
    """
    Classifies texts by checking if they contain any of the specified lemmas.
    """
    doc = nlp(" ".join(self.lemmas))
    lemma_set = set(token.lemma_ for token in doc)
    results = []
    for text in self.texts:
        doc = nlp(text.lower()) if text else nlp("notext")
        text_lemmas = set(token.lemma_ for token in doc)
        results.append(bool(text_lemmas & lemma_set))
    self.pred_scores = results
  # _____________________
  # Supervised machine learning classifier

  def add_SML_params(self,find_learner=False, save_model = False,validation_size=0.30, batch_size=16,
                     learning_rate=2e-5, epochs=4,BERTmodel="bert",preprocess_mode="bert",
                     maxlen=64,max_features = 50000, balance_ratio = 0):
    self.type_ = "supervised"
    self.find_learner=find_learner
    self.save_model = save_model
    self.validation_size=validation_size
    self.batch_size=batch_size
    self.learning_rate=learning_rate
    self.epochs=epochs
    self.BERTmodel=BERTmodel
    self.preprocess_mode=preprocess_mode
    self.maxlen=maxlen
    self.max_features = max_features
    self.balance_ratio = balance_ratio # for undersampling majority class

  def convert_misBinary(self, return_scores = False):
    output = []
    for pred in self.pred_scores:
      if "not" in pred.lower():
        output.append(0)
      else:
        output.append(1)
    if not return_scores:
      self.pred_scores = output
    else:
      return output

  def classify_with_supervised(self):
    # create vectors of texts from loaded dataset
    balance_ratio = self.balance_ratio
    validation_size = self.validation_size
    preprocess_mode = self.preprocess_mode
    sentences = self.texts
    true_scores = self.true_scores

    print(f"s:{len(sentences)},ts:{len(true_scores)}, s(ts):{sum(true_scores)}")
    # Split dataset
    trainText, validText, trainScores, validScores = train_test_split(sentences, true_scores,
                                                                      test_size=validation_size,
                                                                      random_state=10, stratify=true_scores)

    self.MLTrain = pd.DataFrame({"text": trainText, "Misunderstanding": [int(x) for x in trainScores]})
    if balance_ratio > 0:
      df_ = self.MLTrain
      # select misunderstanding turns
      var_df = df_[df_["Misunderstanding"]==1]
      # select non-misunderstanding turns; stating the random state ensures reproducibility
      notVar_df = df_[df_["Misunderstanding"]==0].sample(balance_ratio*len(var_df), random_state=10)
      df_ = pd.concat([var_df,notVar_df])
      self.MLTrain = df_
    self.MLValid = pd.DataFrame({"text": validText, "Misunderstanding": [int(x) for x in validScores]})
    self.true_scores = self.MLValid["Misunderstanding"].to_list()
    self.texts = self.MLValid["text"].to_list()
    # Preprocess texts
    (x_train,  y_train), (x_validation, y_validation), preproc = text.texts_from_df(train_df = self.MLTrain, text_column = "text",
                                                                            label_columns = ["Misunderstanding"], val_df = self.MLValid,
                                                                            preprocess_mode=preprocess_mode, # embeddings to use
                                                                            maxlen=self.maxlen, # max number of words for a document
                                                                            max_features = self.max_features) # size of the network
    # Prime the model
    self.preproc = preproc
    model = text.text_classifier(self.BERTmodel, train_data=(x_train, y_train), preproc=preproc)
    # Create the learner object
    learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=self.batch_size)
    if self.find_learner:
      learner.lr_find()
      learner.lr_plot()
    else:
      learner.fit_onecycle(self.learning_rate, self.epochs)
      self.learner = learner
      # train the model

  def predict_new_texts(self, save_model=False):
    texts_ = self.MLValid
    texts_ = texts_["text"].to_list()
    learner = self.learner
    preproc = self.preproc
    predictor = ktrain.get_predictor(learner.model, preproc)
    preds = predictor.predict(texts_) #,return_proba=True)
    self.predictor = predictor
    self.pred_scores = preds #np.argmax(preds, axis=1)
    self.convert_misBinary()

  def save_SMLmodel(self, wd = os.getcwd()):
    predictor = self.predictor
    predictor.save(wd)

  def return_learner(self):
    return self.learner, self.preproc

  def return_MLValid(self):
    return self.MLValid
  # _____________________
  # Few-shot classifier


  def add_prompt_param(self,GPTmodel,role,prompt,suffix,examples="", nExamples=0):
    self.GPTmodel = GPTmodel
    self.prompt = "\n".join([prompt,examples,suffix])
    self.role = role
    self.examples = examples
    self.nExamples = nExamples
    self.type_ = "few-shot"
    self.cost = 0
    self.total_tokens = 0

  def gptActualCost(self, response):
    '''calculates the gpt cost'''
    engine = self.GPTmodel
    total_tokens=response.usage.total_tokens
    total_tokens_1k_units = total_tokens/1000

    if engine=='gpt-3.5-turbo':
        cost=total_tokens_1k_units*0.0005
    elif engine=='gpt-4-turbo':
        cost=total_tokens_1k_units*0.01
    elif engine=='gpt-4-32k':
        cost=total_tokens_1k_units*0.12
    else:
        print('getCost error: engine not found')
        return
    return cost, total_tokens

  def get_llm_response(self,messages,temperature=0, max_tokens = 100, max_attempts = 3):
    '''
    Function that takes messages format for ChatGPT input and returns the response text.
    '''
    GPTmodel = self.GPTmodel

    for attempt in range(0, max_attempts):
      try:
        #. request timeout ADD IN
        response = openai.chat.completions.create(model=GPTmodel, messages = messages, temperature=temperature, max_tokens=max_tokens)
        response_text = response.choices[0].message.content
        self.LLMScores.append(response_text)
        response_cost, token_count = self.gptActualCost(response)
        self.cost += response_cost
        self.total_tokens += token_count
        break  # If analysis was successful, break out of the retry loop
      except Exception as e:
        print(f"Error processing text on attempt {attempt+1}: {e}")
        if attempt + 1 == max_attempts:
          print(f"Skipping text after {max_attempts} failed attempts.")
          response_text
    return response_text


  def define_messages(self, text_to_classify):
    '''
    Function for creating a basic messages format from a prompt, a role, and a text to classify (all strings)
    '''
    prompt = self.prompt
    role = self.role
    prompt = prompt.format(text_to_classify)
    messages = [{'role': 'system', 'content': role},
                {'role': 'user', 'content' : prompt}]
    return messages

  def convert_llm_scores_binary(self, return_scores = False):
    '''
    Function for converting a string "Yes" or "No" into binary format - used for the clarification requests
    '''
    llm_scores = self.pred_scores
    new_scores = []
    for s in llm_scores:
      if "yes" in s.lower():
        new_scores.append(1)
      else:
        new_scores.append(0)
    if not return_scores:
      self.pred_scores = new_scores
    else:
      return new_scores

  def classify_with_fewshot(self,  max_tokens = 100, max_attempts = 3, temperature = 0):
    '''
    Function for running a prompt over a series of texts (expects a list)
    '''

    prompt = self.prompt
    role = self.role
    input_texts = self.texts
    GPTmodel = self.GPTmodel
    scores = []
    for txt in tqdm(input_texts):
      message = self.define_messages(txt)
      try:
        response = self.get_llm_response(message,temperature=temperature,
                                         max_tokens=max_tokens, max_attempts=max_attempts)
      except:
        response = "Error in response"
      scores.append(response)
    self.pred_scores = scores
    self.convert_llm_scores_binary()


  # _____________________
  # Functions for running classifiers
  def run_classifier(self):
    """
    Determines the type of classification to use and applies it.
    """
    if self.type_ == "rule-based-1":
      self.classify_with_spacy_pattern(nlp)
    elif self.type_ == "rule-based-2":
      self.classify_with_spacy_lemma(nlp)
    elif self.type_ == "supervised":
      self.classify_with_supervised()
      if self.find_learner == False:
        self.predict_new_texts()
    elif self.type_ == "few-shot":
      self.classify_with_fewshot()
      cost = self.cost
      total_tokens = self.total_tokens
      avg_tokens = self.total_tokens / self.train_size
      print(f"This run cost {cost:.2f}$ for {total_tokens} tokens. Average tokens: {avg_tokens:.2f}")
    else:
      raise ValueError("Invalid classifier type specified.")

  def get_model_report(self, display=True):
    true_scores = self.true_scores
    pred_scores = self.pred_scores
    print(f"true:{len(true_scores)},pred:{len(pred_scores)}")
    cm = confusion_matrix(true_scores, pred_scores)
    TP = cm[1, 1]
    FN = cm[1, 0]
    FP = cm[0, 1]
    TN = cm[0, 0]
    report = classification_report(true_scores, pred_scores, output_dict=True)
    F1_var = report['1']['f1-score']  # F1 score for class '1'
    F1_avg = report['weighted avg']['f1-score']  # Weighted average F1 score
    Precision = report['1']['precision']
    Recall = report['1']['recall']
    precision, recall, thresholds = precision_recall_curve(true_scores, pred_scores)
    AUC_PR = auc(recall, precision)
    AUC_ROC = roc_auc_score(true_scores, pred_scores)
    if display:
      print(f'AUC-PR: {AUC_PR:.2f}\n')
      print(f'AUC-ROC: {AUC_ROC:.2f}\n')
      print(classification_report(true_scores, pred_scores))
    else:
      return Precision,Recall,AUC_PR,AUC_ROC,F1_avg,F1_var,TP,TN,FP,FN

  def log_model(self, wandb_project):
    """
    Function runs the model and logs the data for wandb.
    """
    if self.find_learner:
      return
    else:
      Precision,Recall,AUC_PR,AUC_ROC,F1_avg,F1_var,TP,TN,FP,FN = self.get_model_report(display=False)
      type_ = self.type_
      lwb = log_wandb(wandb_project)
      lwb.add_KeyVariables(len(self.texts),Precision,Recall,
                       AUC_PR,AUC_ROC,F1_avg,F1_var,
                       TP,TN,FP,FN,type_)
      if type_ in ["rule-based-1", "rule-based-2"]:
        lwb.add_ClassVars(Terms=0, nTerms=0, PatternOrLemma=0)
      elif type_ == "supervised":
        lwb.add_ClassVars(BERTmodel=self.BERTmodel, validation_size=self.validation_size,
                          learning_rate=self.learning_rate, epochs=self.epochs,
                          batch_size=self.batch_size)

      elif type_ == "few-shot":
        lwb.add_ClassVars(GPTmodel=self.GPTmodel, role=self.role, prompt=self.prompt,
                          examples="", nExamples=0)
      lwb.end_run()

  def return_results(self):
    return self.pred_scores

  def run_and_log(self, wandb_project, display=True):
    self.run_classifier()
    if not self.find_learner:
      self.get_model_report(display)
      self.log_model(wandb_project)
    elif self.find_learner == False:
      self.get_model_report(display)
      self.log_model(wandb_project)

  def get_misclassification(self, return_all = False):
    df = pd.DataFrame({"text":self.texts,"true":self.true_scores,
                       "pred":self.pred_scores})
    # Function to classify each row
    def classify_row(row):
      if row['true'] == 1 and row['pred'] == 1:
        return 'TP'
      elif row['true'] == 0 and row['pred'] == 1:
        return 'FP'
      elif row['true'] == 1 and row['pred'] == 0:
        return 'FN'
      elif row['true'] == 0 and row['pred'] == 0:
        return 'TN'
    df['Classification'] = df.apply(classify_row, axis=1)
    if return_all:
      return df
    else:
      return df[df["Classification"].isin(["FP","FN"])]
  def preprocess_text(self, text):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    words = nltk.word_tokenize(text)
    cleaned_text = [lemmatizer.lemmatize(word.lower()) for word in words if word.isalnum() and word.lower() not in stop_words]
    return ' '.join(cleaned_text)

  def plot_misclassifications(self, df, FN_FP="FN"):
    df['cleaned_text'] = df['text'].apply(self.preprocess_text)
    if FN_FP == "FN":
      print("Word Cloud for False Negatives:")
      texts = " ".join(df[df['Classification'] == 'FN']['cleaned_text'])
    else:
      print("Word Cloud for False Positives:")
      texts = " ".join(df[df['Classification'] == 'FP']['cleaned_text'])
    wordcloud = WordCloud(width = 800, height = 400, background_color ='white').generate(texts)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

  def return_LLMscores(self):
    return self.LLMScores


## RULE BASED EXAMPLE USAGE
#print(f"N lemmas (duplicates removed manual): {len(terms)}")
#RB_lemClass = Classifier_(texts=texts,true_scores=true_scores)
#RB_lemClass.add_rule_based_terms(lemmas, "lemma")
#RB_lemClass.add_rule_based_terms(patterns, "pattern")
#RB_lemClass.run_and_log(wandb_project)

## SUPERVISED CLASSIFIER EXAMPLE USAGE
## Run classifier
#find_learner=False
#validation_size=0.05 # 30 20 10
#batch_size= 128 # 64,32,16
#learning_rate=2e-5# 5e-5, 4e-5, 3e-5, and 2e-5 1e-5
#epochs=4
#BERTmodel="bert"
#preprocess_mode="bert"
#maxlen=30
#max_features = 50000 # 100000 ; 50000 ; 35000   -> 50000 seemed best
#balance_ratio = 0 # rebalances the training data
#SML_Class = Classifier_(texts=texts,true_scores=true_scores)
#SML_Class.add_SML_params(find_learner=find_learner,validation_size=validation_size,
#                         batch_size=batch_size,learning_rate=learning_rate,epochs=epochs,
#                         BERTmodel=BERTmodel,preprocess_mode=preprocess_mode,maxlen=maxlen,
#                         max_features=max_features,balance_ratio=balance_ratio)
#SML_Class.SML_Classifier()
#SML_Class.predict_new_texts()
#SML_Class.run_and_log(wandb_project)



#### Plotting and summary object

This class is used throughout to do various plotting and statistical functions.

In [6]:
class TextDataStats:
  def __init__(self, df, text_column="text", binary_column="Misunderstanding",
               IRR_columns = ["Coder1","Coder2","Coder3","Coder4"],
               group_column = "Round"):
    self.df = df
    self.text_column = text_column
    self.binary_column = binary_column
    self.IRR_columns = IRR_columns
    self.group_column = group_column

  def preprocess_text(self):
    """
    Extracts words and sentences from the text, counts them and adds to the dataframe.
    """
    self.df['words'] = self.df[self.text_column].apply(lambda x: re.findall(r'\b\w+\b', x.lower()))
    self.df['word_count'] = self.df['words'].apply(len)

  def basic_stats(self):
    """
    Computes basic statistics for overall and grouped data.
    """
    self.preprocess_text()

    # General stats
    general_stats = self.df.describe(include=[np.number]).loc[['mean', 'std', 'min', '50%', 'max'], ['word_count']]
    general_stats.rename(index={'50%': 'median'}, inplace=True)
    # Grouped stats by binary column
    grouped_stats = self.df.groupby(self.binary_column).agg({
        'word_count': ['mean', 'median', 'std', 'min', 'max'],
    })
    # Binary column distribution
    binary_dist = self.df[self.binary_column].value_counts(normalize=True).to_frame('distribution')
    return general_stats.round(2), grouped_stats.round(2), binary_dist.round(2)

  def BasicReport(self):
    """
    Generates a report combining all statistics in a readable text format.
    """
    general_stats, grouped_stats, binary_dist = self.basic_stats()

    # Creating a structured text report
    report = "Text Data Statistics Report\n\n"
    report += "General Statistics:\n"
    report += general_stats.to_string() + "\n\n"

    report += "Statistics by Binary Column:\n"
    for name, group in self.df.groupby(self.binary_column):
        report += f"\nGroup: {name}\n"
        report += grouped_stats.loc[name].to_string() + "\n"
    return report


  def get_IRR(self, df):
    """
    Fetches the absolute agreement, Krippendorff's Alpha, Gwet's AC1
    """
    df = df[self.IRR_columns]
    # Get absolute agreement
    df = df.astype(int)
    IRR_out = []
    for i, row in df.iterrows():
      for k in list(df.columns):
        IRR_out.append([k, str(i), row[k]])
    ratingtask = agreement.AnnotationTask(data=IRR_out)
    ags = ratingtask.avg_Ao()
    # Get Gwet AC1 and K Alpha
    cac= CAC(df)
    print(cac)
    #print(cac_4raters)
    Gwet_obj = cac.gwet()
    Alpha_obj = cac.krippendorff()
    return ags, Alpha_obj, Gwet_obj
  def process_object(self, IRR_obj, sig_level = 0.001):
    """
    This processes the cac output for the IRR into two strings for reporting
    """
    s = IRR_obj["est"]["coefficient_value"]
    ci1 = IRR_obj["est"]["confidence_interval"][0]
    ci2 = IRR_obj["est"]["confidence_interval"][1]
    stat_string = f"{s:.2f} CI = ({ci1:.2f}, {ci2:.2f})"
    Z = IRR_obj["est"]["z"]
    pval = IRR_obj["est"]["p_value"]
    if pval < sig_level:
      sig_string = f"z = {Z:.2f}; p < {sig_level}"
    else:
      sig_string = f"z = {Z:.2f}; p = {pval:.3f}"

    return stat_string, sig_string


  def IRRreport(self):
    df = self.df.sort_values([self.group_column])
    rounds = df[self.group_column].unique()
    agreement, alphas_, alpha_sigs_ = [],[],[]
    ac1s, ac1s_sigs, ss = [],[],[]
    for i in rounds:
      tdf = df[df[self.group_column] == i]
      ss.append(len(tdf))
      ags, alpha_, gwets_ = self.get_IRR(tdf)
      agreement.append(ags)
      alpha_stat, alpha_sig = self.process_object(alpha_)
      alphas_.append(alpha_stat)
      alpha_sigs_.append(alpha_sig)

      ac1_stat, ac1_sig = self.process_object(gwets_)
      ac1s.append(ac1_stat)
      ac1s_sigs.append(ac1_sig)

    return pd.DataFrame({"Round":["Round " + str(i) for i in rounds],
                         "Sample size":ss, "Agreement":agreement,
                         "K's Alpha":alphas_,"K's Alpha significance":alpha_sigs_,
                         "Gwet's AC1":ac1s,"Gwet's AC1 significance":ac1s_sigs,
                         })

  def get_disagreements(self,n=10, return_df = False):
    """
    Prints n disagreements for the IRR results
    """
    df = self.df
    disag = []
    for i, row in df.iterrows():
      x = 0
      for coder in self.IRR_columns:
        x += row[coder]
      disag.append(x)
    df["disag"] = disag
    ncoders = len(self.IRR_columns)
    df = df[df["disag"] < ncoders]
    df = df[df["disag"] > 0]
    if return_df:
      return df.round(2)
    else:
      sdf = df.sample(n)
      for s in sdf.text:
        print("----------")
        print(s)

  def get_misclassifications(self, n=5, return_all=False):
    """
    Function to report on the misclassifications across all three classifiers.
    """
    df = self.df

    def classify(row):
      base, fs, sup = int(row["Manual"]), int(row["LLM"]), int(row["supervised"])
      if sup == base:
        return "TP (All)" if base == 1 else "TN (All)" if fs == base else "FP (LLM)" if base == 0 else "FN (LLM)"
      else:
        return "FN (supervised)" if fs == base and base == 1 else "FP (supervised)" if fs == base else "FP (All)" if base == 1 else "FN (All)"

    df["FN_FP"] = df.apply(classify, axis=1)

    if return_all:
      return df

    misclassifications = {
        "FP (All)": df[df.FN_FP == "FP (All)"].text.to_list(),
        "FP (supervised)": df[df.FN_FP == "FP (supervised)"].text.to_list(),
        "FP (LLM)": df[df.FN_FP == "FP (LLM)"].text.to_list(),
        "FN (All)": df[df.FN_FP == "FN (All)"].text.to_list(),
        "FN (supervised)": df[df.FN_FP == "FN (supervised)"].text.to_list(),
        "FN (LLM)": df[df.FN_FP == "FN (LLM)"].text.to_list()
    }

    for key, value in misclassifications.items():
      print(f"--- {key.replace('_', ' ')} count: -- {len(value)}")

    print(f"\nPrinting {n} examples of each classifier type.\n")

    for key, value in misclassifications.items():
      print(f"------ {key.replace('_', ' ').upper()} ------")
      for example in value[:n]:
        print(f"- {example}")
      print("--------")


  def RAMP_plot(self, x_col, y_col, group_col,
                pastel_colors = ['#77B5FE', '#FF6961', '#B19CD9'],
                title="", width=800, height=500, line_width=2, line_opacity=0.5,
                font_size=14, tick_size=12):
    """
    Creates a connected scatter plot with customizable font size and tick size
    """
    self.df[group_col] = self.df[group_col].astype('category')

    scatter_fig = px.line(self.df, x=x_col, y=y_col, color=group_col,
                          title=title, template='plotly_white',
                          labels={x_col: x_col, y_col: y_col, group_col: group_col},
                          markers=True,
                          color_discrete_sequence=pastel_colors)

    for group, group_df in self.df.groupby(group_col):
        min_x = group_df[x_col].min()
        max_x = group_df[x_col].max()
        min_y = group_df[group_df[x_col] == min_x][y_col].iloc[0]
        max_y = group_df[group_df[x_col] == max_x][y_col].iloc[0]
        color_index = group_df[group_col].cat.codes.unique()[0] % len(pastel_colors)
        scatter_fig.add_trace(go.Scatter(
            x=[min_x, max_x],
            y=[min_y, max_y],
            mode='lines',
            name=f'{group} - Range Line',
            line=dict(color=pastel_colors[color_index], width=line_width, dash='dash'),
            opacity=line_opacity,
            showlegend=False))

    # Update layout to include font size and tick size settings
    scatter_fig.update_layout(
        title=dict(text=title, font=dict(size=font_size)),
        xaxis=dict(title=dict(text=x_col, font=dict(size=font_size)),
                   tickfont=dict(size=tick_size)),
        yaxis=dict(title=dict(text=y_col, font=dict(size=font_size)),
                   tickfont=dict(size=tick_size)),
        legend=dict(font=dict(size=font_size)),
        width=width, height=height
    )

    scatter_fig.show()



# 1. Manual coding

The first stage of RAMP is a manual coding stage, where a codebook is developed through the process of training coders and conducting small pilot studies. We use Krippendorff's Alpha ([Krippendorff, 1970](https://journals.sagepub.com/doi/10.1177/001316447003000105)) for quantifying the inter-rater reliability of coders.


This section reports the inter-rater reliability of these studies and the final inter-rater reliability on a shared dataset. The shared dataset was coded blind, with coders unaware of which sentences were being shared and which were exclusive to the individual.

## 1.1 Input:

### 1.1.1 Data


The raw dataset contains sentences from online dialogues, sampled from three sources:

**Reddit conversations from 27 subreddits**.

> This data was downloaded using the Reddit API by the authors.

**Twitter Customer Support data**  ([Thought Vector & Axelbrooke, 2017](https://www.kaggle.com/datasets/thoughtvector/customer-support-on-twitter)).

> This data was downloaded from (Copyright: CC BY-NC-SA 4.0).

**Wikipedia Talk Pages data** ([Danescu-Niculescu-Mizil et al., 2012](https://convokit.cornell.edu/documentation/wiki.html)).

> This data was downloaded using Cornell University's [ConvoKit](https://convokit.cornell.edu) Python package (Copyright: CC BY 4.0)

**Notes:**

> All author names and sentences have been anonymized following ethical guidelines for the study.
> As a further precaution, the sentences are shuffled and the source (e.g., Reddit, Twitter) removed from the dataframe.

In [7]:
# Manual coded dataset final size
print(f"Full dataset size: {len(train) + len(validation)}")

Full dataset size: 21994


In [8]:
# Explanation of shared subsamples manual coding
tdf = St1Through
prev_rounds = tdf[tdf.Round!=6]
IRR_sample = tdf[tdf.Round==6]
IRR_texts = IRR_sample.text.to_list()
non_IRR_texts = prev_rounds.text.to_list()
crossover_texts = [t for t in IRR_texts if t in non_IRR_texts]
crossover_df = tdf[tdf.text.isin(crossover_texts)].drop_duplicates()
crossover_df.loc[:,"All_coders"] = crossover_df.Coder1 + crossover_df.Coder2 + crossover_df.Coder3 + crossover_df.Coder4
crossover_df.loc[:,"All_coders"] = crossover_df["All_coders"].apply(lambda x: 1 if x > 1 else 0)

all_rounds_len = len(tdf)
n_crossovers_all = all_rounds_len - len(tdf.text.drop_duplicates())
pct_all_cross = round((n_crossovers_all/all_rounds_len)*100,2)
IRR_len = len(IRR_sample)
n_crossovers = len(crossover_texts)
pct_cross= round((n_crossovers/IRR_len)*100,2)
n_mis_in_cross = crossover_df.All_coders.sum()
pct_mis = round((n_mis_in_cross/n_crossovers)*100,2)
print(f"""
There are {all_rounds_len} sentences across all rounds of coding.
{n_crossovers_all} ({pct_all_cross}%) sentences were shared across various rounds.

The IRR set (Round 6) contains {IRR_len} sentences.
Of these sentences, {n_crossovers} ({pct_cross}%) appeared in another round.
Of these crossover sentences, {n_mis_in_cross} ({pct_mis}%) were coded for misunderstanding.
""")

IRR_df_no_crossovers = IRR_sample[~IRR_sample.text.isin(crossover_texts)]


There are 6099 sentences across all rounds of coding.
288 (4.72%) sentences were shared across various rounds.

The IRR set (Round 6) contains 1551 sentences.
Of these sentences, 135 (8.7%) appeared in another round.
Of these crossover sentences, 17 (12.59%) were coded for misunderstanding.



In [9]:
# Get the IRR from the final round (validation)
IRR_final = St1Through[St1Through["Round"]==6]
IRR_through = St1Through[St1Through["Round"]!=6]

## 1.2 Throughput

Inter rater reliability across four pilot studies coding random samples of sentences:

In [10]:
tds = TextDataStats(IRR_through)
tds.IRRreport().round(2)

<irrCAC.raw.CAC Subjects: 689, Raters: 4, Categories: [0, 1], Weights: "identity">
<irrCAC.raw.CAC Subjects: 1178, Raters: 4, Categories: [0, 1], Weights: "identity">
<irrCAC.raw.CAC Subjects: 1068, Raters: 4, Categories: [0, 1], Weights: "identity">
<irrCAC.raw.CAC Subjects: 783, Raters: 4, Categories: [0, 1], Weights: "identity">
<irrCAC.raw.CAC Subjects: 830, Raters: 4, Categories: [0, 1], Weights: "identity">


,Round,Sample size,Agreement,K's Alpha,K's Alpha significance,Gwet's AC1,Gwet's AC1 significance
0,Round 1,689,0.95,"0.57 CI = (0.49, 0.66)",z = 13.47; p < 0.001,"0.94 CI = (0.93, 0.96)",z = 129.52; p < 0.001
1,Round 2,1178,0.97,"0.69 CI = (0.62, 0.77)",z = 17.45; p < 0.001,"0.97 CI = (0.96, 0.98)",z = 251.29; p < 0.001
2,Round 3,1068,0.97,"0.72 CI = (0.66, 0.79)",z = 21.62; p < 0.001,"0.96 CI = (0.95, 0.97)",z = 199.78; p < 0.001
3,Round 4,783,0.94,"0.78 CI = (0.73, 0.82)",z = 34.19; p < 0.001,"0.92 CI = (0.91, 0.94)",z = 100.61; p < 0.001
4,Round 5,830,0.98,"0.75 CI = (0.68, 0.82)",z = 20.70; p < 0.001,"0.97 CI = (0.97, 0.98)",z = 230.60; p < 0.001


We can see that the Alpha gets progressively better.

We can also see the deceptive nature of absolute agreement. For instance, the low alpha of 0.57 in the first round has 95% agreement is because coders were generally good at recognizing *not* misunderstandings but bad at agreeing on what sentences were misunderstandings. The problem is caused by the skewed nature of the dataset (misunderstandings only 8% of data).


We ended the training at Round 5, as the agreement diminishes from the previous round.

## 1.3 Output

In [11]:
tds = TextDataStats(IRR_final)
tds.IRRreport().round(2)

<irrCAC.raw.CAC Subjects: 1551, Raters: 4, Categories: [0, 1], Weights: "identity">


,Round,Sample size,Agreement,K's Alpha,K's Alpha significance,Gwet's AC1,Gwet's AC1 significance
0,Round 6,1551,0.98,"0.79 CI = (0.74, 0.84)",z = 29.76; p < 0.001,"0.98 CI = (0.97, 0.98)",z = 330.29; p < 0.001


This is very good agreement (98%) with moderate inter-rater reliability (Krippendorff's Alpha  = 0.79).


To sense-check the inter-rater reliability, we can remove all sentences that appeared in the previous rounds, leaving us with only sentences that the coders had yet to score before:

In [12]:
tds = TextDataStats(IRR_df_no_crossovers)
tds.IRRreport().round(2)

<irrCAC.raw.CAC Subjects: 1416, Raters: 4, Categories: [0, 1], Weights: "identity">


,Round,Sample size,Agreement,K's Alpha,K's Alpha significance,Gwet's AC1,Gwet's AC1 significance
0,Round 6,1416,0.98,"0.78 CI = (0.73, 0.84)",z = 27.16; p < 0.001,"0.98 CI = (0.97, 0.98)",z = 314.46; p < 0.001


# 2. Computation

This stage reports the development of three classifiers and their testing on the validation data. The development stage reports the accuracy statistics across 21 different attempts to improve the classifiers' performance on the training data.

The three classifiers are:

1. A rule-based dictionary classifier

This classifier labels a text as misunderstandings if it identifies any of a pre-defined set of words (the dictionary).

We use this for binary classification. However, it can be used for producing a ratio or frequency count of the words. In this case, a ratio is pointless as the short sentences almost never contain two words relating to misunderstandings. The frequency count will mostly be 1 or 0, and therefore a binary classification. Ratios offer more information for longer texts, as these would generate more word counts.  

2. A supervised machine learning classifier

This classifier fine-tunes a BERT ([Devlin et al., 2019](https://arxiv.org/abs/1810.04805)) model using the ktrain packages. We also plot the increasing accuracy from the development stage as we explored the use of different parameters.

3. A large language model (LLM) classifier

This classifier sends a prompt to GPT-4o (version May 13, 2024) alongside the text to label. It's response is then processed into a binary classification. This is also known as zero-shot or few-shot classification, named after how many empirical examples are included in the prompt ([Brown et al., 2020](https://arxiv.org/abs/2005.14165)).



## 2.1 Input

In [15]:
# Information on the validation data = binary column is misunderstandings.
tds = TextDataStats(validation)
print(tds.BasicReport())

Text Data Statistics Report

General Statistics:
        word_count
mean         14.97
std          11.71
min           0.00
median       12.00
max         203.00

Statistics by Binary Column:

Group: 0
word_count  mean       14.90
            median     12.00
            std        11.68
            min         0.00
            max       203.00

Group: 1
word_count  mean      15.82
            median    12.00
            std       12.09
            min        2.00
            max       80.00



These are the spaCy terms for the rule-based classifier:

In [ ]:
terms = [{"label":"MISUNDERSTANDING","pattern":[{"LOWER":{"IN":["what","why"]}},{"OP":"*","POS":"AUX"},{"POS":"VERB","OP":"*"},{"IS_PUNCT":True,"OP":"?"}]},
          {"label":"MISUNDERSTANDING","pattern":[{"LOWER":"what"},{"LOWER":"do"},{"LOWER":"you"},{"LOWER":"mean"},{"IS_PUNCT":True,"OP":"?"}]},
            {"label":"MISUNDERSTANDING","pattern":[{"LOWER":"could"},{"LOWER":"you"},{"LEMMA":{"IN":["elaborate","expand","explain"]}},{"OP":"?","IS_PUNCT":True}]},
             {"label":"MISUNDERSTANDING","pattern":[{"LOWER":"i"},{"LOWER":"don't"},{"LEMMA":"understand"}]},
              {"label":"MISUNDERSTANDING","pattern":[{"LOWER":{"IN":["sorry","pardon","excuse"]}},{"LOWER":"me"},{"LOWER":"could"},{"LOWER":"you"},{"LOWER":"repeat"},{"OP":"?","IS_PUNCT":True}]},
               {"label":"MISUNDERSTANDING","pattern":[{"LOWER":"are"},{"LOWER":"you"},{"LOWER":"saying"},{"IS_ALPHA":True,"OP":"+"},{"OP":"?","IS_PUNCT":True}]},
                {"label":"MISUNDERSTANDING","pattern":[{"LEMMA":{"IN":["misinterpret","misunderstand","misconstrue"]}},{"POS":"ADP"},{"IS_ALPHA":True,"OP":"+"}]},
                 {"label":"MISUNDERSTANDING","pattern":[{"LOWER":{"IN":["did","do","does"]}},{"LOWER":"you"},{"LEMMA":"mean"},{"OP":"?","IS_PUNCT":True}]},
                  {"label":"MISUNDERSTANDING","pattern":[{"LOWER":"let's"},{"LOWER":"talk"},{"LOWER":"about"},{"IS_ALPHA":True,"OP":"+"}]},
                   {"label":"MISUNDERSTANDING","pattern":[{"LOWER":"to"},{"LOWER":"clarify"},{"OP":"?","IS_PUNCT":True}]}]

This is the pre-trained BERT model for the supervised classifier (trained on 90% of the training data - the remaining 10% were used to monitor its accuracy).

In [ ]:
predictor = ktrain.load_predictor('supervised_model')

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


This is the prompt for the LLM classifier:

In [ ]:
role = """
*Role* You are a research assistant tasked with identifying whether a sentence indicates a misunderstanding.
*Misunderstanding definition* A misunderstanding occurs during dialogue when one participant has an incorrect understanding of another’s perspective.
"""
prompt = """
There are two categories of misunderstanding:
1. “Direct” misunderstandings. These occur when a participant evidences a misunderstanding of another participant’s point.
2. “Felt” misunderstandings. These occur when a participant feels their previous turn was misunderstood by another participant.
This is a non-exhaustive list of possible sentences indicating misunderstanding.
1. Explicit statement: The sentence explicitly indicates the speaker doesn't understand another’s perspective (e.g., "I don't get what you're trying to say about the dog")
2. Clarification question: The question seeks to clarify the other’s perspective (e.g., "What do you mean?")
3. Request for confirmation: A question that seeks confirmation on the other’s understanding of the speaker’s previous turn(e.g., "You really think that I meant all dogs?")
4. Correction of Other: Correcting another speaker’s misunderstanding of the present speaker’s previous turn(e.g., "You've misunderstood my point", “You don’t get it.”)
5. Clarification or apology about speaker's intentions: Clarifying the meaning of what the speaker previously said (e.g., "Sorry, I meant to say X")
6. Misunderstanding due to lack of response (e.g., "Why did you change the subject?")
7. Editing a message at a later time: This is when a speaker in text-based dialogue comes back to edit their comment after the fact (e.g., "EDIT": That's what I said)
Here are some examples of sentences indicating misunderstandings:
- Jane, that article was what I was talking about.
- Why not go further? - Do you think that was ok?
- I apologise for saying that, but I meant the other stuff.
- @John But when? - @John Please tell me why I've been stuck here for so long.
- What drove that thought? - I actually said "sure thing".
- You serious?
- I'm not sure what I could have done differently.
TASK:
Does the below sentence indicate a possible misunderstanding?
Only respond with "Yes" or "No"
Sentence: {}
Response:"""

## 2.2 Throughput

These two plots show the classifier performance according to (1) Matthews Correlation Coefficient (MCC) (2) Weighted F1 for the classifier.

Each point indicates a change in the input parameters of the classifier. For the rule-based classifier, this was adding and altering the words. For the supervised classifier, this was altering the hyper-parameters. For the few-shot classifier, this was changing the prompt.

In [16]:
def matthews_correlation_coefficient(tp, tn, fp, fn):
    numerator = (tp * tn) - (fp * fn)
    denominator = ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) ** 0.5
    if denominator == 0:
        return 0  # Undefined MCC, return 0 as a safe default
    return numerator / denominator

In [17]:
target_cols = ["Order","Classifier","F1_avg","F1_var","AUC_PR", "AUC_ROC","Precision","Recall","FP","FN","TP","TN"]
ThroughRes = St2Through[target_cols].round(2).sort_values("AUC_PR", ascending = False)
ThroughRes["Matthews Correlation Coefficient"] = ThroughRes.apply(lambda row: matthews_correlation_coefficient(row["TP"],row["TN"],row["FP"],row["FN"]), axis=1)
ThroughRes = ThroughRes.rename(columns={"F1_avg": "Weighted F1"})
tdf = ThroughRes.sort_values(by=["Order","Classifier"])

In [18]:
tdf = tdf.replace({"few-shot":"LLM"})
tds = TextDataStats(tdf)
tds.RAMP_plot("Order", "Matthews Correlation Coefficient","Classifier", width=1100,height=700, font_size = 20, tick_size=17)

/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:1971: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(required_grouper, sort=False)  # skip one_group groupers
/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:1992: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])
<ipython-input-6-2ea9dbe75de7>:225: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [19]:
tds.RAMP_plot("Order", "Weighted F1","Classifier", width=1100,height=700)

/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:1971: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:1992: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

<ipython-input-6-2ea9dbe75de7>:225: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



We can call up the best classifiers of each type, alongside any relevant input parameters. The rule-based words are the ones defined in Section 2.1.

In [20]:
# Best supervised classifier and parameters
St2Through["MCC"] = St2Through.apply(lambda row: matthews_correlation_coefficient(row["TP"],row["TN"],row["FP"],row["FN"]), axis=1)
supdf = St2Through[["Order","Classifier","F1_avg","AUC_PR", "MCC","Precision","Recall","validation_size","epochs","learning_rate","batch_size"]]
supdf[supdf["Classifier"]=="supervised"].sort_values("MCC", ascending = False).head(1).round(2)

,Order,Classifier,F1_avg,AUC_PR,MCC,Precision,Recall,validation_size,epochs,learning_rate,batch_size
61,20,supervised,0.94,0.7,0.65,0.74,0.62,0.3,4.0,0.0,128.0


In [21]:
# Best rule-based classifier (using lemma list)
rbdf = St2Through[["Order","Classifier","F1_avg","AUC_PR","MCC", "Precision","Recall","PatternOrLemma","train_size","nTerms"]]
rbdf[rbdf["Classifier"]=="rule-based"].sort_values("MCC", ascending = False).head(1).round(2)

,Order,Classifier,F1_avg,AUC_PR,MCC,Precision,Recall,PatternOrLemma,train_size,nTerms
34,14,rule-based,0.87,0.32,0.22,0.33,0.24,pattern,14728,10.0


In [22]:
# Best LLM classifiers (using prompt)
rbdf = St2Through[["Order","Classifier","F1_avg","AUC_PR", "MCC","Precision","Recall", "train_size","GPTmodel"]]
rbdf[rbdf["Classifier"]=="few-shot"].sort_values("MCC", ascending = False).head(1).round(2)

,Order,Classifier,F1_avg,AUC_PR,MCC,Precision,Recall,train_size,GPTmodel
12,13,few-shot,0.91,0.58,0.51,0.54,0.59,1000,gpt-4-turbo


## 2.3 Output

### 2.3.1 Classification reports

The below prints the classification reports from our final classifiers on the validation dataset.

In [23]:
texts = out.text.to_list()
true_scores = out.Misunderstanding.to_list()
pred_scores = out["rule-based"].to_list()
rbClassifier = Classifier(texts, true_scores)
rbClassifier.add_pred_scores(pred_scores)
rbClassifier.get_model_report()

AUC-PR: 0.31

AUC-ROC: 0.61

MCC: 0.22

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      5909
           1       0.29      0.27      0.28       511

    accuracy                           0.89      6420
   macro avg       0.62      0.61      0.61      6420
weighted avg       0.89      0.89      0.89      6420



In [ ]:
# Rule-based classifier - unhash to replicate the analysis
#out = validation.copy()
#texts = out.text.to_list()
#true_scores = out.Misunderstanding.to_list()
#rbClassifier = Classifier(texts, true_scores)
#rbClassifier.add_rule_based_terms(terms, 'pattern')
#rbClassifier.run_classifier()
#rbClassifier.get_model_report()

In [24]:
true_scores = out.Misunderstanding.to_list()
pred_scores = out["supervised"].to_list()
smlClassifier = Classifier(texts, true_scores)
smlClassifier.add_pred_scores(pred_scores)
smlClassifier.get_model_report()

AUC-PR: 0.73

AUC-ROC: 0.88

MCC: 0.69

              precision    recall  f1-score   support

           0       0.98      0.96      0.97      5909
           1       0.65      0.79      0.71       511

    accuracy                           0.95      6420
   macro avg       0.81      0.88      0.84      6420
weighted avg       0.95      0.95      0.95      6420



In [ ]:
# For supervised machine learning classifier
#smlClassifier = Classifier(texts, true_scores)
#smlClassifier.add_SML_classifier(predictor)
#smlClassifier.run_classifier()
#smlClassifier.get_model_report()

In [ ]:
true_scores = out.Misunderstanding.to_list()
pred_scores = out["few-shot"].to_list()
fsClassifier = Classifier(texts, true_scores)
fsClassifier.add_pred_scores(pred_scores)
fsClassifier.get_model_report()

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


AUC-PR: 0.56

AUC-ROC: 0.80

MCC: 0.47

              precision    recall  f1-score   support

           0       0.97      0.91      0.94      5909
           1       0.39      0.69      0.50       511

    accuracy                           0.89      6420
   macro avg       0.68      0.80      0.72      6420
weighted avg       0.93      0.89      0.90      6420



In [ ]:
# LLM classifier
#gpt_model = "gpt-4o"
#fsClassifier = Classifier(texts, true_scores)
#fsClassifier.add_few_shot_classifier(gpt_model, prompt, role)
#fsClassifier.run_classifier()
#fsClassifier.get_model_report()

In [ ]:
#out["rule-based"] = rbClassifier.pred_scores
#out["supervised"] = smlClassifier.pred_scores
#out["few-shot"] = fsClassifier.pred_scores
#out.to_csv("RAMP_Stage2Output_v2.csv",index=False)

# 3. Evaluation

This section looks at disagreements and misclassifications in order to inform the final stage of RAMP. These are used to infer surprising findings from which to identify potential problems of construct and concept validity.

This analysis is qualitative and is informed by the below disagreements and misclassifications

## 3.1 Disagreements evaluation

In [ ]:
# Get sample of disagreements
tds = TextDataStats(IRR_final)
tds.get_disagreements(25)

----------
In any case, everyone has different things they find satisfying to do on Wikipedia; why don't you spend time on things that give you pleasure?
----------
@Ask_Spectrum I've gave your company enough of my patience ive had enough, you just lost a customer!.
----------
Update: as a few have pointed out, the term racist was a poor choice of words.
----------
this is amc, but i feel you
----------
Well, I'm not talking about Western Sahara specifically.
----------
We wouldn't be able to comment further than what was discussed yesterday, until Omniserve come back.
----------
The rest not so rightÔ£ø√º√≤√ë thanks for correcting me!
----------
The article is actually a lot better and resourceful than it originally appeared but I believe my edits have improved it, even if I picked up a few horses in Jutland rather than Jutland horse and probably needed minor copyedits.
----------
Why go from zero to 100 today?
----------
I mean is there proof for that third one?
----------
You seem t

## 3.2 Misclassifications evaluation

In [ ]:
out = out.rename(columns={"Misunderstanding":"Manual", "few-shot":"LLM"})

In [ ]:
tds = TextDataStats(out)
misdf = tds.get_misclassifications(return_all = True)

In [ ]:
# For printing rule-based misclassifications - these are fairly arbitrary
for i in misdf[misdf["Manual"]==1][misdf["rule-based"]==0].sample(10).text.values:
  print(i)

btw I'm not Ronald!
hehe, whoops!
No news is good news?
@Company_Handle For the type of issue reported I thought I may have at least been contacted for some information.
Do you steal from seniors too or just kids?
Am I missing something?
Isn't this to show pics we've taken??
We don't think it is.
Like John hasn't exhibited a consistent attitude conducive to collaboration?
Oh, I see.


<ipython-input-17-3b1cc0282d4c>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for i in misdf[misdf["Manual"]==1][misdf["rule-based"]==0].sample(10).text.values:


In [ ]:
# For supervised and few shot classifiers
tds = TextDataStats(out)
tds.get_misclassifications(n=25)

--- FP (All) count: -- 53
--- FP (supervised) count: -- 111
--- FP (LLM) count: -- 436
--- FN (All) count: -- 111
--- FN (supervised) count: -- 53
--- FN (LLM) count: -- 0

Printing 25 examples of each classifier type.

------ FP (ALL) ------
- Sure, do you want me to?
- This is completely unacceptable and to worsen it is that you lot do not respond quick enough.
- I am very patient as to what-will-happen-next, although I don't seem to manage it with spur-of-the-moment replies (oops).
- I'm not really committed to extensive rewriting as it's only wiki and can be edited anyway.
- I take that back.
- hahahahaha my apologies
- How can you fame a Wikipedian?
- @Company_Handle @Company_Handle What type of ticket have you bought?
- Again, Industry Canada is a reliable source, even if not the preferred one, and those numbers are much better than no numbers at all.
- Anything you can do to make him believe that I mean no ill will would be very helpful.
- For one i didn't even know the person \

### 3.2.1 Using LIME to explore BERT classifier

In [ ]:
# For assessing the supervised classifier FN and FP
predictor.explain("It's me, again")

In [ ]:
predictor.explain("Mark, the experience you are describing is something we'd never do.")

In [ ]:
# For assessing the supervised classifier FN and FP
predictor.explain("Not sure what that is")

In [ ]:
# For assessing the supervised classifier FN and FP
predictor.explain("But... They do.")

# 4. Conclusions

In the manual coding stage, we had acceptable inter-rater reliability (Krippendorff's Alpha = 0.79) following 5 training rounds.
In the computational classification stage, we created three different text classifiers, one rule-based, one supervised, and one LLM. Overall, the supervised machine learning classifier – a fine-tuned BERT model – is much better than both the LLM and rule-based classifiers. The classifier's performance is acceptable (MCC = 0.69) with room for improvement.

 When troubleshooting the results, we can see that the false negatiives are missing some key clarification questions (e.g., "So what about everyone else?"). We can also see that the classifiers are picking up on "new information" questions, not directed at another's perspeective (e.g., "Are you having this issue with any other channels?"). The misclassifications indicate that the classifier is generally struggling with edge cases more than standard cases.

# 5. References


> Brown, T., Mann, B., Ryder, N., Subbiah, M., Kaplan, J. D., Dhariwal, P., Neelakantan, A., Shyam, P., Sastry, G., Askell, A., Agarwal, S., Herbert-Voss, A., Krueger, G., Henighan, T., Child, R., Ramesh, A., Ziegler, D., Wu, J., Winter, C., … Amodei, D. (2020). Language models are few-shot learners. Advances in Neural Information Processing Systems, 33, 1877–1901. https://proceedings.neurips.cc/paper/2020/hash/1457c0d6bfcb4967418bfb8ac142f64a-Abstract.html

> Danescu-Niculescu-Mizil, C., Lee, L., Pang, B., & Kleinberg, J. (2012). Echoes of power: Language effects and power differences in social interaction. Proceedings of the 21st International Conference on World Wide Web, 699–708. https://doi.org/10.1145/2187836.2187931

> Devlin, J., Chang, M.-W., Lee, K., & Toutanova, K. (2019). BERT: Pre-training of deep bidirectional transformers for language understanding. Proceedings of the 2019 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies, Volume 1 (Long and Short Papers), 4171–4186. https://doi.org/10.18653/v1/N19-1423

> Honnibal, M., Montani, I., Van Landeghem, S., & Boyd, A. (2022). SpaCy: Industrial-Strength Natural Language Processing in Python. Explosion.

> Korobov, M. (Presenter). (2017). Explaining behavior of Machine Learning models with eli5 library. EuroPython.

> Korobov, M., & Lopuhin, K. (2024). eli5: Debug machine learning classifiers and explain their predictions (0.13.0) [Python; OS Independent]. https://github.com/eli5-org/eli5

> Krippendorff, K. (1970). Estimating the reliability, systematic error and random error of interval data. Educational and Psychological Measurement, 30(1), 61–70. https://doi.org/10.1177/001316447003000105

> Maiya, A. S. (2022). ktrain: A low-code library for augmented machine learning (arXiv:2004.10703). arXiv. https://doi.org/10.48550/arXiv.2004.10703

> OpenAI, Achiam, J., Adler, S., Agarwal, S., Ahmad, L., Akkaya, I., Aleman, F. L., Almeida, D., Altenschmidt, J., Altman, S., Anadkat, S., Avila, R., Babuschkin, I., Balaji, S., Balcom, V., Baltescu, P., Bao, H., Bavarian, M., Belgum, J., … Zoph, B. (2024). GPT-4 Technical Report (arXiv:2303.08774). arXiv. https://doi.org/10.48550/arXiv.2303.08774

> Pennebaker, J. W., Francis, M. E., & Booth, R. J. (2001). Linguistic Inquiry and Word Count: LIWC. Mahway: Lawrence Erlbaum Associates, 71(2001).

> Ribeiro, M. T., Singh, S., & Guestrin, C. (2016). ‘Why should I trust you?’: Explaining the predictions of any classifier (arXiv:1602.04938). arXiv. https://doi.org/10.48550/arXiv.1602.04938

> Shivan, B., & Chaitanya, A. (2024). textstat: Calculate statistical features from text (0.7.3) [Python]. https://github.com/shivam5992/textstat



# Export notebook:

In [ ]:
# Install necessary packages
!sudo apt-get install texlive-xetex texlive-fonts-recommended texlive-plain-generic

In [ ]:
# First MANUALLY download locally to the working directory
# Convert the downloaded file to an HTML file
!jupyter nbconvert --to PDF "RAMP_CaseStudy_12May2024_v31.ipynb"

[NbConvertApp] Converting notebook RAMP_CaseStudy_12May2024_v31.ipynb to PDF
/usr/local/lib/python3.10/dist-packages/nbconvert/filters/datatypefilter.py:41: UserWarning: Your element with mimetype(s) dict_keys(['text/html']) is not able to be represented.
  warn(
[NbConvertApp] Writing 178109 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 163015 bytes to RAMP_CaseStudy_12May2024_v31.pdf
